In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pprint
import pylab as pl
import numpy as np

# Limit rows disp# Limit rows displayed in notebook
pd.set_option('display.max_rows', 20)
pd.set_option('display.precision', 2)

In [2]:
import json
import pandas
 
def convertBusiness(x):
    ob = json.loads(x)
    return {"business_id":ob["business_id"], "stars":ob["stars"], "city":ob["city"]}

def convertReviews(x):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    ob = json.loads(x)
    return {"business_id":ob["business_id"], "stars":ob["stars"], "date":ob["date"]}

def createPandas(func, json_filename):
    result=[]
    f=open(json_filename, 'r')
    for line in f:
        result.append(func(line))
    return pd.DataFrame(result)

print "Creating Business Pandas"
pd_business = createPandas(convertBusiness,'yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json')
print "Creating Review Pandas"
pd_review = createPandas(convertReviews, 'yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json')
print "Completed Creating Pandas"


Creating Business Pandas
Creating Review Pandas
Completed Creating Pandas


In [3]:
#preprocess the business data
pd_business_lasvegas = pd_business[pd_business["city"] == "Las Vegas"]
#remove the unwanted stars data present in business.json
pd_business_lasvegas.pop('stars')

pd_review_lasvegas = pd.merge(pd_business_lasvegas, pd_review, left_on='business_id', right_on='business_id', how='inner')
print list(pd_review_lasvegas.columns.values)
gb = pd_review_lasvegas.groupby(['business_id'],as_index=False)
#print gb.size()
gb = gb.agg({'date' : np.min})
gb["new"] = (gb["date"] >= "2011-01-01")
#print gb

['business_id', 'city', 'date', 'stars']


In [4]:
pd_lasvegas = pd.merge(pd_review_lasvegas, gb, left_on='business_id', right_on='business_id', how='inner')
pd_lasvegas.rename(columns={'date_y': 'start_date'}, inplace=True)
#print pd_lasvegas

In [5]:
pd_old_business_vegas = pd_lasvegas[pd_lasvegas['new'] == False]
pd_new_business_vegas = pd_lasvegas[pd_lasvegas['new'] == True]
#print pd_old_business_vegas
#print pd_new_business_vegas

In [18]:
import datetime
import time
global count
count=0
#calculate the mean change in the ratings for all old business given a start date of one new business
def find_mean_change(start_date,time_gap,count):

    from_start_date=pd.to_datetime(pd.to_datetime(start_date)-time_gap)
    to_after_date=pd.to_datetime(pd.to_datetime(start_date)+time_gap)
    pd_old_business_vegas['before']=((pd_old_business_vegas['date_x'] < start_date) & (pd.to_datetime(pd_old_business_vegas['date_x']) > from_start_date))
    
    pd_old_business_vegas['after']=((pd_old_business_vegas['date_x'] > start_date) & (pd.to_datetime(pd_old_business_vegas['date_x']) < to_after_date))
    
    pd_old_before=pd_old_business_vegas[pd_old_business_vegas['before']==True]
    pd_old_after=pd_old_business_vegas[pd_old_business_vegas['after']==True]
    #print pd_old_after

    gb_before = pd_old_before.groupby(['business_id'], as_index=False)
    gb_before = gb_before.agg({'stars' : np.mean})

    gb_after=pd_old_after.groupby(['business_id'], as_index=False)
    gb_after=gb_after.agg({'stars' : np.mean})

    before_after_ratings=pd.merge(gb_before, gb_after, left_on='business_id', right_on='business_id', how='inner')
    before_after_ratings.rename(columns={'stars_x': 'before_rating'}, inplace=True)
    before_after_ratings.rename(columns={'stars_y': 'after_rating'}, inplace=True)
    before_after_ratings['diff']=before_after_ratings['after_rating']-before_after_ratings['before_rating']
    #print before_after_ratings

    mean=before_after_ratings[['diff']].mean()
    #print mean
    count=count+1
    return (mean, count)

#calculate mean for every new business
time_gap=datetime.timedelta(days=365)
count=0
mean=0
print pd_new_business_vegas.shape
start_time = time.time()
for index, row in pd_new_business_vegas.iterrows():
    mean_new, count_new=find_mean_change(row['start_date'],time_gap,count)
    mean=mean+mean_new
    count=count_new
print mean/count
print time.time()-start_time
    

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(164700, 6)
diff   -6.7e-02
dtype: float64
25346.9849999
